In [24]:
import numpy as np
import pandas as pd
import vectorbt as vbt
import yfinance as yf

# Parameters
num = 10
metric1 = "total_return"
metric2 = "max_drawdown"
start = "2020-01-01"

# Download TSLA data from Yahoo Finance
df = yf.download("BTC-USD", auto_adjust=True, start=start)
df = df[['Close']]

# Generate parameter arrays for moving averages entry/exit points
arr1 = np.linspace(20, 50, num=num)
arr2 = np.linspace(50, 80, num=num)

short = list(arr1.astype('i'))
long = list(arr2.astype('i'))

# Create a mesh grid in order to simulate more MA combinations
grid1 = np.array(np.meshgrid(short, long)).T.reshape(-1, 2)
lst1 = [x[0] for x in grid1[:, [0]]]
lst2 = [x[0] for x in grid1[:, [1]]]

# Generate short-term and long-term moving averages for each combination
short_ma = vbt.MA.run(df, lst1)
long_ma = vbt.MA.run(df, lst2)

# Generate RSI
rsi = vbt.RSI.run(df)

# Define entry and exit points for RSI
entry_points = np.linspace(30, 45, num=num)
exit_points = np.linspace(70, 110, num=num)

# Create a mesh grid in order to simulate more RSI combinations
grid2 = np.array(np.meshgrid(entry_points, exit_points)).T.reshape(-1, 2)

# Creating rsi indicators
rsi_buy_signal = rsi.rsi_crossed_above(list(grid2[:, [0]]))  
rsi_sell_signal = rsi.rsi_crossed_below(list(grid2[:, [1]])) 

ma_buy_signal = short_ma.ma_crossed_below(long_ma)
ma_sell_signal = short_ma.ma_crossed_below(long_ma)

entries = rsi_buy_signal.vbt & ma_buy_signal
exits = rsi_sell_signal.vbt & ma_sell_signal

print(entries)

# Build the portfolio using the combined signals
pf = vbt.Portfolio.from_signals(df, entries, exits)

# Display results
print(pf.stats())

# Get the performance metric
pf_perf = pf.deep_getattr(metric1)
print(pf_perf)

pf_perf_matrix = pf_perf.vbt.unstack_to_df(index_levels = ["rsi_crossed_above", 2], 
                  column_levels = ["rsi_crossed_below", 3] )


pf_perf_matrix.vbt.heatmap( 
    xaxis_title = "exit",
    yaxis_title = "entry" ).show()


[*********************100%%**********************]  1 of 1 completed


rsi_crossed_above   30.0                                                   \
ma_window             20                                                    
ma_window             50     53     56     60     63     66     70     73   
Date                                                                        
2020-01-01         False  False  False  False  False  False  False  False   
2020-01-02         False  False  False  False  False  False  False  False   
2020-01-03         False  False  False  False  False  False  False  False   
2020-01-04         False  False  False  False  False  False  False  False   
2020-01-05         False  False  False  False  False  False  False  False   
...                  ...    ...    ...    ...    ...    ...    ...    ...   
2024-09-20         False  False  False  False  False  False  False  False   
2024-09-21         False  False  False  False  False  False  False  False   
2024-09-22         False  False  False  False  False  False  False  False   

c:\Users\gokub\ex\journey\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\gokub\ex\journey\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Users\gokub\ex\journey\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Users\gokub\ex\journey\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

C:\Users\gokub\AppData\Local\Temp\ipykernel_21860\675554737.py:58: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x0000023017FA84A0>. Pass column to select a single column/group.



Start                         2020-01-01 00:00:00
End                           2024-09-25 00:00:00
Period                                       1729
Start Value                                 100.0
End Value                              216.028033
Total Return [%]                       116.028033
Benchmark Return [%]                   792.966662
Max Gross Exposure [%]                       61.0
Total Fees Paid                               0.0
Max Drawdown [%]                         48.54579
Max Drawdown Duration                  475.803279
Total Trades                                 0.63
Total Closed Trades                          0.02
Total Open Trades                            0.61
Open Trade PnL                         106.320799
Win Rate [%]                                100.0
Best Trade [%]                         453.009471
Worst Trade [%]                         93.617556
Avg Winning Trade [%]                  273.313514
Avg Losing Trade [%]                          NaN
